# Doc2Vec Tutorial on the Lee Dataset

In [1]:
import gensim
import os
import collections
import smart_open
import random

## What is it?

Doc2Vec is an NLP tool for representing documents as a vector and is a generalizing of the Word2Vec method. This tutorial will serve as an introduction to Doc2Vec and present ways to train and assess a Doc2Vec model.

## Resources

* [Word2Vec Paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [Doc2Vec Paper](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)
* [Dr. Michael D. Lee's Website](http://faculty.sites.uci.edu/mdlee)
* [Lee Corpus](http://faculty.sites.uci.edu/mdlee/similarity-data/)
* [IMDB Doc2Vec Tutorial](https://github.com/piskvorky/gensim/blob/develop/docs/notebooks/doc2vec-IMDB.ipynb)

## Getting Started

To get going, we'll need to have a set of documents to train our doc2vec model. In theory, a document could be anything from a short 140 character tweet, a single paragraph (i.e., journal article abstract), a news article, or a book. In NLP parlance a collection or set of documents is often referred to as a <b>corpus</b>. 

For this tutorial, we'll be training our model using the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included in gensim. This corpus contains 314 documents selected from the Australian Broadcasting
Corporation’s news mail service, which provides text e-mails of headline stories and covers a number of broad topics.

And we'll test our model by eye using the much shorter [Lee Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) which contains 50 documents.

In [2]:
# Set file names for train and test data
test_data_dir = '/Users/usr0102377/gitrepo/Reuters-full-data-set'

my_train_file = test_data_dir + os.sep + 'split_titleaa'
my_test_file = test_data_dir + os.sep + 'split_titleab'

## Define a Function to Read and Preprocess Text

Below, we define a function to open the train/test file (with latin encoding), read the file line-by-line, pre-process each line using a simple gensim pre-processing tool (i.e., tokenize text into individual words, remove punctuation, set to lowercase, etc), and return a list of words. Note that, for a given file (aka corpus), each continuous line constitutes a single document and the length of each line (i.e., document) can vary. Also, to train the model, we'll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply the zero-based line number.

In [3]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [4]:
train_corpus = list(read_corpus(my_train_file))
test_corpus = list(read_corpus(my_test_file, tokens_only=True))

Let's take a look at the training corpus

In [5]:
train_corpus[:2]

[TaggedDocument(words=[u'update', u'tsmc', u'plans', u'five', u'new', u'advanced', u'wafer', u'plants', u'paper'], tags=[0]),
 TaggedDocument(words=[u'roadie', u'gets', u'moment', u'in', u'the', u'spotlight'], tags=[1])]

And the testing corpus looks like this:

In [6]:
print(test_corpus[:2])

[[u'research', u'and', u'markets', u'cf', u'dry', u'eye', u'syndrome', u'forecast', u'and', u'market', u'analysis', u'to'], [u'zumasys', u'joins', u'second', u'harvest', u'food', u'bank', u'of', u'orange', u'county', u'in', u'fight', u'against', u'hunger']]


Notice that the testing corpus is just a list of lists and does not contain any tags.

## Training the Model

### Instantiate a Doc2Vec Object 

Now, we'll instantiate a Doc2Vec model with a vector size with 50 words and iterating over the training corpus 55 times. We set the minimum word count to 2 in order to give higher frequency words more weighting. Model accuracy can be improved by increasing the number of iterations but this generally increases the training time. Small datasets with short documents, like this one, can benefit from more training passes.

In [8]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)
gensim.models.doc2vec.Doc2Vec?

### Build a Vocabulary

In [9]:
model.build_vocab(train_corpus)  # takes roughly 1-2 minutes

Essentially, the vocabulary is a dictionary (accessible via `model.wv.vocab`) of all of the unique words extracted from the training corpus along with the count (e.g., `model.wv.vocab['penalty'].count` for counts for the word `penalty`).

### Time to Train

If the BLAS library is being used, this should take no more than 3 seconds.
If the BLAS library is not being used, this should take no more than 2 minutes, so use BLAS if you value your time.

In [ ]:
#model.train?
%time model.train(train_corpus, total_examples=model.corpus_count)#, epochs=model.iter)
# started a while ago, now 15:18. I will do this on the server in parallel.

### Inferring a Vector

One important thing to note is that you can now infer a vector for any piece of text without having to re-train the model by passing a list of words to the `model.infer_vector` function. This vector can then be compared with other vectors via cosine similarity.

In [10]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([-0.04635895, -0.08925428, -0.05811016, -0.01581211, -0.04152944,
        0.01337591, -0.12117779,  0.06591314, -0.00748183,  0.06647876,
       -0.16933711, -0.02466986, -0.02944544,  0.03836808,  0.06411615,
       -0.07434633, -0.02737885,  0.03957457, -0.00055505,  0.04093107,
        0.00086868,  0.0437365 , -0.12501197,  0.07140061,  0.03730097,
        0.02794078,  0.02039102,  0.08718826,  0.11064955, -0.00487213,
       -0.08063051, -0.05700861,  0.005919  ,  0.022075  , -0.13468936,
        0.04974887,  0.10404179,  0.08159174, -0.0363216 ,  0.08685723,
       -0.02877273, -0.07175135, -0.01315776,  0.08636736,  0.03127725,
        0.1136056 , -0.07934703, -0.00866038, -0.02839444,  0.05480706], dtype=float32)

## Assessing Model

To assess our new model, we'll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we're pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we've likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we'll keep track of the second ranks for a comparison of less similar documents. 

In [11]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

Let's count how each document ranks with respect to the training corpus 

In [12]:
collections.Counter(ranks)  # Results vary due to random seeding and very small corpus

Counter({0: 291, 1: 9})

Basically, greater than 95% of the inferred documents are found to be most similar to itself and about 5% of the time it is mistakenly most similar to another document. the checking of an inferred-vector against a training-vector is a sort of 'sanity check' as to whether the model is behaving in a usefully consistent manner, though not a real 'accuracy' value.

This is great and not entirely surprising. We can take a look at an example:

In [13]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Notice above that the most similar document is has a similarity score of ~80% (or higher). However, the similarity score for the second ranked documents should be significantly lower (assuming the documents are in fact different) and the reasoning becomes obvious when we examine the text itself

In [14]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus))

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (9): «some roads are closed because of dangerous conditions caused by bushfire smoke motorists are being asked to avoid the hume highway between picton road and the illawarra highway where police have reduced the speed limit from kilometres an hour to in southern sydney picton road is closed between wilton and bulli appin road is closed from appin to bulli tops and all access roads to royal national park are closed motorists are also asked to avoid the illawarra highway between the hume highway and robertson and the great western highway between penrith and springwood because of reduced visibility in north western sydney only local residents are allowed to use wisemans ferry road and upper color road under police escort»

Similar Document (46, 0.7000304460525513): «seven yachts have been forced to retire from the sydney to hobart yacht race after storm hit the fleet off the new south wales south coast overnight south australian yacht sap ausmaid and secret men business b

## Testing the Model

Using the same approach above, we'll infer the vector for a randomly chosen test document, and compare the document to our model by eye.

In [15]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus))
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (23): «china said sunday it issued new regulations controlling the export of missile technology taking steps to ease concerns about transferring sensitive equipment to middle east countries particularly iran however the new rules apparently do not ban outright the transfer of specific items something washington long has urged beijing to do»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (36, 0.5387957692146301): «new york mayor rudolph giuliani today bade farewell to the city he has led for the past eight years calling for soaring beautiful memorial to be built on the site of the world trade centre mr giuliani speaking just four days ahead of his final day in office further defended his tough stance on crime and homelessness time magazine person of the year also praised new york diversity as the source of strength pulling the city through the difficult aftermath of september deadly attacks this place has to be sanctified mr giuliani said t

### Wrapping Up

That's it! Doc2Vec is a great way to explore relationships between documents.